In [2]:
import numpy as np
import math
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.translate.metrics import alignment_error_rate
from nltk.metrics.scores import (precision, recall)
from operator import itemgetter

In [21]:
#DATA
file = open('./train.en', 'r')
train_en = file.read()
raw_sentences_train_en = train_en.split("\n")

file = open('./train.hi', 'r')
train_hi = file.read()
raw_sentences_train_hi = train_hi.split("\n")

file = open('./test.en', 'r')
test_en = file.read()
raw_sentences_test_en = test_en.split("\n")

file = open('./test.hi', 'r')
test_hi = file.read()
raw_sentences_test_hi = test_hi.split("\n")

file = open('./dev.en.txt', 'r')
dev_en = file.read()
raw_sentences_dev_en = dev_en.split("\n")

file = open('./dev.hi', 'r')
dev_hi = file.read()
raw_sentences_dev_hi = dev_hi.split("\n")

In [26]:
def add_null(data):
    list_ = []
    for sentence in data:
        list_.append("NULL " + sentence)
#     print(list_[2])
    return list_

In [28]:
sentences_dev_hi = add_null(raw_sentences_dev_hi)
sentences_test_hi = add_null(raw_sentences_test_hi)
sentences_train_hi = add_null(raw_sentences_train_hi)

In [24]:
def make_lower_case(data):
    list_ = []
    for sentence in data:
        list_.append("NULL " + sentence.lower())
#     print(list_[2])
    return list_

In [25]:
sentences_dev_en = make_lower_case(raw_sentences_dev_en)
sentences_test_en = make_lower_case(raw_sentences_test_en)
sentences_train_en = make_lower_case(raw_sentences_train_en)

In [6]:
def is_converged(new, old, epoch):
    epsilone = 0.00000001
#     new = list(new.values())
#     old = list(old.values())
    
#     new_val = []
#     for (hin, ddict) in new.items():
#         for (eng, prob) in ddict.items():
#             new_val.append('{0:.8f}'.format(prob))
            
#     old_val = []
#     for (hin, ddict) in old.items():
#         for (eng, prob) in ddict.items():
#             old_val.append('{0:.8f}'.format(prob))        

#     for i in range(len(old_val)):
#         if math.fabs(float(new_val[i]) - float(old_val[i])) > epsilone: 
    if epoch < 15:
        return False
    return True

In [56]:
def perform_EM(en_sentences, hi_sentences):
    
    uni_ini = 0.00001
    
    translation_prob = defaultdict(lambda: float(uni_ini))
    translation_prob_prev = defaultdict(float)
    
    epoch = 0
    
    while True:

        epoch += 1
        print("epoch num:", epoch,"\n")
        count = defaultdict(float)
        total = defaultdict(float)
        s_total = defaultdict(float)
        
        for index_sen, hin_sen in enumerate(hi_sentences):
            #compute normalization
            hin_sen_words = hin_sen.split(" ")
            for hin_word in hin_sen_words:
                s_total[hin_word] = 0
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        s_total[hin_word] += translation_prob[(hin_word, eng_word)]
            
            #collect counts
            for hin_word in hin_sen_words:
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        count[(hin_word, eng_word)] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]
                        total[eng_word] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]                   

        #estimate probabilities
        for (hin_word, eng_word) in translation_prob.keys():
                translation_prob[(hin_word, eng_word)] = count[(hin_word, eng_word)]/total[eng_word]

        if is_converged(translation_prob, translation_prob_prev, epoch) == True:
            break
        
        translation_prob_prev = translation_prob
        
        
    return translation_prob


In [8]:
def train_model(sentences_train_en, sentences_train_hi):
    
    translation_prob = perform_EM(sentences_train_en, sentences_train_hi)
    return translation_prob


In [57]:
tef = train_model(sentences_train_en, sentences_train_hi)

epoch num: 1 

epoch num: 2 

epoch num: 3 

epoch num: 4 

epoch num: 5 

epoch num: 6 

epoch num: 7 

epoch num: 8 

epoch num: 9 

epoch num: 10 

epoch num: 11 

epoch num: 12 

epoch num: 13 

epoch num: 14 

epoch num: 15 



In [ ]:
# np.save("./models/IBMmodel1tef_3", tef)

In [58]:
iterations = 0
for ((e_word, hin_word), value) in sorted(tef.items(), key=itemgetter(1), reverse=True):
    if iterations < 20:
        print("{:<20}{:>20.2}".format("P(%s|%s)" %(e_word, hin_word), value))
    else:
        break
    iterations += 1

P(रिंक|skatin)                       1.0
P(महादेव|mahadeva)                   1.0
P(गंगा|ganges)                      0.99
P(राजस्थान|rajasthan)                0.99
P(भूटान|bhutan)                     0.99
P(भवन|bhavan)                       0.99
P(दिल्ली|delhi)                     0.99
P(ऐतिहासिक|historical)                0.99
P(महादेव|mahadev)                   0.99
P(हजार|thousand)                    0.99
P(आज|today)                         0.99
P(बस्तर|bastar)                     0.99
P(पटना|patna)                       0.99
P(मिजोरम|mizoram)                   0.99
P(धर्मशाला|dharamshala)                0.99
P(नेहरू|nehru)                      0.99
P(नदी|river)                        0.99
P(जैन|jain)                         0.98
P(नाम|name)                         0.98
P(बंगाल|bengal)                     0.98


In [59]:
def translate_sentence(sentence, tef, file):
    '''
    takes the best translation of an hindi word
    and appends to eng sentence
    '''
    eng_sentence = []
#     print("hin:",sentence)
    
    tokens = sentence.split(" ")
    for idx, token in enumerate(tokens):
        probabilities = []
        eng_words = []
#         print("token id, token", idx, token)
        max_score = -1
        max_eng_word = ""
        for k, v in tef.items():
            if token == k[0]:
                probabilities.append(v)
                eng_words.append(k[1])
#             print(f'{k[0]} - {v}')
#         probabilities = list(tef[token].values())
#         eng_words = list(tef[token].keys())
        for tef_index, prob in enumerate(probabilities):
            if prob > max_score:
                max_score = prob
                max_eng_word = eng_words[tef_index]
        
        eng_sentence.append(max_eng_word)
    
    eng_sentence = " ".join(eng_sentence)
#     print("eng:", eng_sentence)
    file.write(eng_sentence)
    file.write("\n")
    
    return eng_sentence


In [54]:
def test_model(dataset, tef):
#     tef = np.load('./models/IBMmodel1tef.npy')
    file = open("dev_translations.txt", 'w+')
    translated_data = []
    for sentence in dataset:
        translation = translate_sentence(sentence, tef, file)
        translated_data.append(translation)
    
    file.close()
    return translated_data

In [55]:
predicted_translations = test_model(raw_sentences_test_hi, tef)

hin: ताजा साँसें और चमचमाते दाँत आपके व्यक्तित्व को निखारते हैं ।
eng: fresh skips and shimmering tooth beggar amenity shukla enhance are .
hin: दाँतों से आपका आत्मविश्‍वास भी बढ़ता है ।
eng: teeth from crave confidence also triglisrides is .
hin: हमारे मसूढ़ों और दाँतों के बीच बैक्टीरिया मौजूद होते हैं ।
eng: our initiation and teeth के beach bacteria present kalal are .
hin: ये दाँतों को गंदा और साँसों को बदबूदार बना देते हैं ।
eng: skiiing teeth shukla dirty and aromas shukla stinky dehradun-chakrata deduct are .
hin: यहाँ दिए कुछ आसान नुस्खों की मदद से आप अपने दाँतों को स्वच्छ और साँसों को ताजा रख सकते हैं ।
eng: here preached few easy prescriptions ki help from puzzles cd teeth shukla clean and aromas shukla fresh uddiyaana instantaneous are .
hin: दाँतों को ठीक से साफ करें ।
eng: teeth shukla alright from clean pilates .
hin: दाँतों को ठीक से साफ करने में दो से तीन मिनट का समय लगता है ।
eng: teeth shukla alright from clean doing in two from three minutes bhonsle time held is .
hi

eng:  के overspreading symptoms what are 
hin: कुछ दिनों से हल्का फीवर रहता है ।
eng: few days from lightish pericarditis stays is .
hin: बुखार की जाँच करें और अन्य जरूरी जाँच के पश्चात ही कालाजार प्रमाणित हो सकता है ।
eng: fever ki test pilates and other important test के pronged barhait  certified occur can is .
hin: मेटासिन की गोली देने से बुखार उतर जाता है और फिर 103 तक बुखार चढ़ जाता है ।
eng:  ki pill giving from fever derails known is and descension 103 till fever 12,300 known is .
hin: एक्सरे , टीसी-डीसी नार्मल है ।
eng: x-ray ,  injected is .
hin: चार - चार घंटे पर फीवर को मापें ।
eng: four - four hours on pericarditis shukla monitering .
hin: अगर 100 से ज्यादा फीवर हो तो पैरासीटामोल की गोली दें और किसी औषधि विशेषज्ञ चिकित्सक से परामर्श लें ।
eng: if 100 from gnerally pericarditis occur then  ki pill let and someone medicine specialist therapist from adivsable grind .
hin: एच.आई.वी. क्या है ?
eng: h what is ?
hin: यह कैसे होता है ?RD_PUNC
eng: this how fibre is 
hin: बचाव के उ

eng: many people researcher october in maldives ki  samvat water के underwater ki scanning toronto shukla might barhait  understood occur , but maldives ahsrama held is that if nature bhonsle parbat nandikeshwar ours robbed jacuzzi then world shatrunji  occur kaajikund .
hin: वहाँ की सुंदरता में कुछ ऐसा ही खास है ।
eng: therefrom ki robustness in few bhagirathji barhait particular is .
hin: उस छोटे से देश में मानो इतनी खूबसूरती समाती नहीं ।
eng: vadpradaka younger from country in venus dripping beautifully  not .
hin: बाजार की उस सैर से वापस अपने क्रूज पर जाने के लिए जेट्‍टी की तरफ लौटते हुए , जब निगाहें बरबस सड़क के साथ-साथ चलते समंदर के पानी में गई तो अहसास हुआ कि यहाँ के लोग इस खूबसूरती की कीमत किस हद तक समझते हैं ।
eng: bazaar ki vadpradaka stroll from reverted cd cruise on going के for  ki baagmati returning curvy , when preying involuntarily sarak के hommocystin alchu imbibe के water in scanning then acknowledges happened that here के people this beautifully ki price frss extent t

eng: many days from delhi from outside going ki plan dehradun-chakrata constables were but work ki busyness के alchu this possible not occur relied remained was .
hin: बचपन से वाइल्ड लाइफ का शौक था तो सोचा कि क्यों न इस बार जिम कार्बेट जाया जाए ।
eng: childhood from beasts kotigaon bhonsle hobby was then kedaarnath that why neither this times gym corbett ahmedanagar corpulence .
hin: 2006 में लद्दाख में खारदूंगला पास से आगे पेगोंग लेक तक की बाइकिंग के बाद यह दूसरा बाइक टूर था ।
eng: 2006 in ladakh in  aalam from ahead  dimna till ki biking के after this tirathdev bike atique was .
hin: 27 दिसंबर की तड़के हम दिल्ली से जिम कार्बेट के लिए रवाना हुए , लेकिन जिस जोशखरोश से निकले थे कोहरे ने हमारी रफ्तार रफूचक्कर कर दी ।
eng: 27 december ki bunking we delhi from gym corbett के for ustoli curvy , but dignified  from scraf were fog researcher ours industrialisation  disgrace sharbari .
hin: हापुड़ से गजरौला के बीच का सफर वाकई थकाऊ था ।
eng: hapur from gajraula के beach bhonsle journey really  wa

In [11]:
def find_recall_precision(predicted_translations):
    y_pred = list(predicted_translations)
    y_true = sentences_dev_hi
    recall = recall(y_true, y_pred)
    precision = precision(y_true, y_pred)
    return recall, precision;